# Homework 3, Basic: Part 3, PageRank

### Part 3: PageRank (Worth 50 points)

Recall that PageRank can be modeled using matrix operations as follows.  Let $M$ be a _weight transfer matrix_ in which:

$M[i,j] = \frac{1}{n_j}$, if $n_j > 0$ and 

$M[i,j] = 0$ otherwise

where page $i$ is pointed to by page $j$ and page $j$ has $n_j$ outgoing links. And define a _dampening factor_ $\alpha = 0.85$ and a corresponding $\beta = 1 - \alpha$.  Initialize the PageRank vector

$PR^{(0)}=[1,1,1,\ldots]^T$

(i.e., a matrix with m rows by 1 column, filled with ones).  Then we can compute the PageRank $PR$ for each iteration as:

$PR^{(i)}= \alpha \cdot M \cdot PR^{(i-1)} + \beta \cdot [1,1,1,\ldots]^T$

### Step 3.1 Download a Web Graph
The following code retrieves a web graph from https://snap.stanford.edu/data/web-NotreDame.txt.gz, which is a reasonably sized Web crawl done by Notre Dame University, and extracts it into `web-NotreDame.txt`.  Run the program to acquire your Web graph.


In [1]:
# Download and decompress data into your Jupyter environment

import urllib.request
import io
import gzip


for file in ['web-NotreDame.txt']:
    print ('Downloading compressed image of', file)
    source = urllib.request.urlopen("https://snap.stanford.edu/data/" + file + ".gz")
    compressedFile = io.BytesIO(source.read())
    decompressedFile = gzip.GzipFile(fileobj=compressedFile)

    with open(file, 'wb') as outfile:
        outfile.write(decompressedFile.read())
        outfile.close()
        print ('Saved', file)


Saved web-NotreDame.txt


### Step 3.2 Load the Notre Dame Web Graph into a Matrix

Next, write Python code to take the data from `web-NotreDame.txt`, read and parse the rows in a Pandas DataFrame (not a Spark DataFrame!).  Restrict the node IDs to values less than 10,000.

_Hints: If you use_ `read_csv`_, you may need to look at the_ `sep` _and_ `skiprows` _options.  Also take a look at the raw data and make sure you know how many rows don't contain data, and how the items are separated.  

In [2]:
# TODO: In this cell, store the data from web-NotreDame.txt in graph_df
# Worth 10 points
import pandas as pd

graph_df = pd.read_csv('web-NotreDame.txt',skiprows = 3,sep = '\t')
graph_df = graph_df.rename(columns = {'# FromNodeId':'FromNodeId'})
bool1 = graph_df['FromNodeId'] < 10000 
bool2 = graph_df['ToNodeId'] < 10000
graph_df = graph_df[bool1 & bool2]
graph_df

,FromNodeId,ToNodeId
0,0,0
1,0,1
2,0,2
3,0,3
4,0,4
5,0,5
6,0,6
7,0,7
8,0,8
9,0,9


In [3]:
if graph_df.shape[1] != 2:
    raise ValueError('Incorrect number of columns')

In [4]:
graph_df.shape

(37841, 2)

Create a weight transfer matrix M corresponding to the Web graph, with edges whose weights are scaled as per the PageRank definition of a weight transfer matrix.  This will form an input into your PageRank algorithm.  Note that the dataset already includes node IDs that go from $0,\ldots,m$, so you can directly use the node IDs as indices in your matrix.  You should not use for loops, and instead, use the DataFrame and array functions that Pandas and NumPy provide as they are much more efficient.

When building $M$, you may need to build some "auxiliary" data structures to speed up performance, e.g., to quickly look up weights associated with node edges.  Note that lookup in an array is typically faster than lookup in a DataFrame.  Finally, you might want to use the_ `apply` _function for Pandas DataFrames or Numpy Matrices as they are orders of magnitude faster than trying to iterate through every row.  However, this is not a requirement -- just make sure you aren’t using for loops!_

In [5]:
graph_df

,FromNodeId,ToNodeId
0,0,0
1,0,1
2,0,2
3,0,3
4,0,4
5,0,5
6,0,6
7,0,7
8,0,8
9,0,9


In [6]:

# TODO: Create the M matrix in this cell
# Worth 20 points

# Create outgoinglinks array

import numpy as np

def get_weight(x):
    return 1/x

graph_df['freq'] = graph_df.groupby('FromNodeId')['FromNodeId'].transform('count')
graph_df['weights'] = graph_df['freq'].apply(get_weight)

arrays = graph_df.values
fromnode = arrays[:,0].astype(int)
tonode = arrays[:,1].astype(int)
weights = arrays[:,3]

M = np.zeros((10000, 10000))
M[tonode, fromnode] = weights





In [7]:
graph_df

,FromNodeId,ToNodeId,freq,weights
0,0,0,17,0.058824
1,0,1,17,0.058824
2,0,2,17,0.058824
3,0,3,17,0.058824
4,0,4,17,0.058824
5,0,5,17,0.058824
6,0,6,17,0.058824
7,0,7,17,0.058824
8,0,8,17,0.058824
9,0,9,17,0.058824


In [8]:
M[10:30,10:30]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0

In [9]:
if M.shape != (10000,10000):
    raise ValueError("Incorrect Matrix dimensions")

### Step 3.3 Compute Matrix-Based PageRank
Implement a function `pagerank(M, alpha, num_iter)` that, when given a square $m \times m$ transition matrix $M$ from Step 3.2, initializes the PageRank vector to $m$ 1’s, sets $\alpha$ = `alpha`, sets $\beta$ appropriately given $\alpha$, and iterates `num_iter` times.  Return an $m$-element vector that consists of the final PageRank scores.

In [10]:
# TODO:
# Write your pagerank function into this cell
# Worth 15 points
# M = np.array([[0,0,1/3,0],[1/2,0,1/3,0],[1/2,0,0,1],[0,1,1/3,0]])

def pagerank(M, alpha, num_iter):
    m = len(M)
    ones = np.array([1]*m)
    pr = ones
    beta = 1 - alpha
    for i in range(num_iter):
        m = len(M)
        pr = alpha * M @ pr + beta * ones
    return pr

    

In [11]:
pr = pagerank(M, 0.85, 15)

pr

array([2.24702638e+02, 2.79759111e+01, 1.14034108e+01, ...,
       1.57231541e-01, 1.57231541e-01, 1.57231541e-01])

Output a DataFrame called `best_pages_df` with the schema `(id, pagerank)` containing the original IDs and PageRanks of the 10 nodes with highest PageRank, in descending order.

In [12]:
# TODO:
# Output 10 tuples using this cell.
# Worth 5 points plus validates your pagerank
pages_df = pd.DataFrame(pr)    
pages_df = pages_df.sort_values(0,ascending=False).reset_index()
best_pages_df = pages_df.rename(columns = {'index':'id',pages_df.columns[1]:'pagerank'})
best_pages_df = best_pages_df[:10]
best_pages_df

,id,pagerank
0,0,224.702638
1,1973,189.250314
2,1790,53.438593
3,1828,50.954873
4,1,27.975911
5,238,26.779136
6,140,23.520898
7,14,22.232264
8,16,21.591054
9,162,18.283386


In [13]:
if best_pages_df.columns[0] != 'id' or best_pages_df.columns[1] != 'pagerank':
    raise ValueError('Incorrect column names')

In [14]:
if best_pages_df.shape[0] != 10:
    raise ValueError('There should be 10 rows in best_pages_df')

In [15]:
if len(np.where(best_pages_df['id'] == 0)[0]) != 1:
    raise ValueError('')